## Purpose  
Demonstrate how leakage can be detected using a data randomization process.  
## Process
Create fake data using bernoulli GLM data-generating process (DGP). Intentionally leak information and then show that this code helps detect the leakage.

In [1]:
import pandas as pd
import sys
import numpy as np
import os
import math
from sklearn import linear_model
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn import metrics 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')

In [2]:
# Set seed
np.random.seed(0)

In [3]:
def randomize(filepath, index_col = None, do_not_randomize = None):
    '''
    Randomize column values of a file. Each column is randomized independently.
    
    Inputs:
        filepath (str): path to file to randomize; may be of type csv or txt
        index_col (str): optional name of column to use as index; will not 
            be randomized
        do_not_randomize (list): optional list of strings indicating names of 
            columns that should not be randomized
    Outputs:
        df (dataframe): dataframe representation of randomized data
        Output file will be generated, named as original file name + "_randomized"
    '''
    
    # Treat csv and txt differently
    filename, file_extension = os.path.splitext(filepath)
    if file_extension == '.csv':
        sep = ","
    elif file_extension == '.txt':
        sep = "\t"
        
    # Randomize column by column
    df = pd.read_csv(filepath, sep = sep, index_col = index_col) 
    
    if do_not_randomize:
        cols = [c for c in df.columns if c not in do_not_randomize]
    else:
        cols = df.columns
        
    for col in cols:
        print('... Randomizing column ' + col)
        df[col] = np.random.permutation(df[col])
        
    # Print to new csv or txt
    new_file = filename + '_randomized' + file_extension
    df.to_csv(new_file)   
    
    return df

In [4]:
def generate_data(period, n_rows, m_columns, threshold, betas, intercept):
    '''
    Generate fake data for a classification task, based on data generating process where:
    - Features are drawn iid from standard normal distribution
    - Regression equation: y = intercept + beta_1 * x1 + .. + beta_m * xm + error
    - Error is drawn from standard normal distribution
    - Outcome is 1 if p value from logistic function exceeds threshold, 0 otherwise
    
    Inputs:
        period (int): time period
        n_rows (int): number of observations to generate
        m_columns (int): number of features to generate
        betas (list): list of numerical values to use as coefficients; length must equal m_columns, 
            i.e. don't include intercept
        intercept (int): constant number to use as intercept of regression equation
        
    Outputs:
        X (dataframe): matrix of feature variables
        y (series): outcome variable
    '''
    print("Generating {} observations...".format(n_rows))
    
    # generate column names
    colnames = []
    for i in range(m_columns):
        colnames.append('x' + str(i + 1))
    
    # generate fake data
    df = pd.DataFrame(np.random.normal(size=(n_rows, m_columns)), columns = colnames)
    df.insert(loc = 0, column = 'intercept', value = intercept)
    betas = [1] + betas
    df['z'] = np.multiply(np.array(betas), df).sum(axis=1) + np.random.normal()
    df['pr'] = df['z'].apply(lambda x: 1/(1 + math.exp(x)))
    df['outcome'] = np.where(df['pr'] > threshold, 1, 0)
    df.index.name = 'idx'
    df['period'] = period
    
    return df

In [5]:
def run_logreg(X_train, y_train, X_test, y_test):
    '''
    Run logistic regression on given test and training sets        
    '''
    print("Baseline accuracy: " + str(round(1 - np.mean(y_train), 2)))
    
    # Create a logistic regression object
    logreg = linear_model.LogisticRegression()

    # Train model with training set
    logreg.fit(X_train, y_train)

    # Make predictions with test set
    y_pred = logreg.predict(X_test)

    # Save probabilities
    probs = logreg.predict_proba(X_test)

    # Print accuracy and AUC
    print("Accuracy: " + str(metrics.accuracy_score(y_test, y_pred)))
    print("AUC Score: " + str(metrics.roc_auc_score(y_test, probs[:, 1])))

    # Print confusion matrix
    print(metrics.confusion_matrix(y_test, y_pred))
    
    return logreg, X_test, y_test, y_pred

In [6]:
def get_X_y(df):
    '''
    Split data frame into X and y objects 
    '''
    X = df.drop(['z', 'pr', 'outcome'], axis = 1)
    y = df['outcome']
    return X, y

def logreg_pipeline(df, temporal = False):
    '''
    Put raw inputs through basic machine learning pipeline.
    
    Inputs:
        dataframe: Pandas dataframe representing raw input file
        temporal (booL): Indicator for whether a temporal TimeSeriesSplit
            should be use instead of non-temporal train_test_split
    OUtputs:
        list of logistic regression model objects
    '''
    
    # Save results of run_logreg
    models = []
    
    # Split into train and test sets
    if temporal: # do temporal train-test split
        periods = np.sort(df.period.unique())
        splits = TimeSeriesSplit(n_splits = max(periods))
        print("Time series split")
        for train_index, test_index in splits.split(periods):
            print("Train periods: ", train_index, "Test periods: ", test_index)
            df_train = df[df.period.isin(train_index)]
            df_test = df[df.period.isin(test_index)]
            X_train, y_train = get_X_y(df_train)
            X_test, y_test = get_X_y(df_test)
            models.append(run_logreg(X_train, y_train, X_test, y_test))

    else: # do random, non-temporal train-test split
        print("Train test split -- test size = 0.33")
        X, y = get_X_y(df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        models.append(run_logreg(X_train, y_train, X_test, y_test))
    
    return models

## Case 1: Stable DGP 
### Non-temporal example with stable DGP; data leakage through outcome variable as a feature

Generate data for 5 periods with constant DGP

In [7]:
frames = []
for i in range(5):
    period_df = generate_data(period = i, n_rows = 2000, m_columns = 5, threshold = 0.8, betas = [1,2,3,4,5], intercept = 1)
    frames.append(period_df)
df = pd.concat(frames)
df.to_csv('fake_data.csv')

Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...


Run original input file through pipeline.

In [8]:
models = logreg_pipeline(df)

Train test split -- test size = 0.33
Baseline accuracy: 0.6
Accuracy: 0.976363636364
AUC Score: 0.998165881132
[[1911   42]
 [  36 1311]]


Put the input file through randomization process, then run it through the pipeline. Model performs no better than random, as expected.

In [9]:
df_random = randomize('fake_data.csv', 'idx', ['period', 'outcome']) # creates csv named fake_data_randomized.csv
models = logreg_pipeline(df_random)

... Randomizing column intercept
... Randomizing column x1
... Randomizing column x2
... Randomizing column x3
... Randomizing column x4
... Randomizing column x5
... Randomizing column z
... Randomizing column pr
Train test split -- test size = 0.33
Baseline accuracy: 0.6
Accuracy: 0.591212121212
AUC Score: 0.525692665258
[[1951    0]
 [1349    0]]


Try introducing an obvious case of data leakage by using the outcome column as a feature. Run this through the pipeline. Model performs well, as expected.

In [10]:
df['leaky_col'] = df['outcome']
models = logreg_pipeline(df)

Train test split -- test size = 0.33
Baseline accuracy: 0.59
Accuracy: 1.0
AUC Score: 1.0
[[1973    0]
 [   0 1327]]


Put the input file through randomization process. Again, introduce data leakage into the pipeline. Model performs better than random, thus suggesting the existence of data leakage.

In [11]:
df_random = randomize('fake_data.csv', 'idx', ['period', 'outcome']) # generates csv named "fake_data_randomized.csv"
df_random['leaky_col'] = df_random['outcome']
models = logreg_pipeline(df_random)

... Randomizing column intercept
... Randomizing column x1
... Randomizing column x2
... Randomizing column x3
... Randomizing column x4
... Randomizing column x5
... Randomizing column z
... Randomizing column pr
Train test split -- test size = 0.33
Baseline accuracy: 0.6
Accuracy: 1.0
AUC Score: 1.0
[[1958    0]
 [   0 1342]]


## Case 2: Stable DGP
### Non-temporal example with stable DGP; data leakage through selection of variables that are highly correlated with outcome (i.e. proxy variables)

Generate data with 1000 variables for 5 periods with constant DGP. Betas are random integers between 0 and 10.

In [12]:
frames = []
for i in range(5):
    period_df = generate_data(period = i, n_rows = 2000, m_columns = 1000, threshold = 0.8, betas = list(np.random.randint(0, 10, size=1000)), intercept = 1)
    frames.append(period_df)
df = pd.concat(frames)
df.to_csv('fake_data.csv')

Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...


Run original input file through pipeline.

In [13]:
models = logreg_pipeline(df)

Train test split -- test size = 0.33
Baseline accuracy: 0.5
Accuracy: 0.815454545455
AUC Score: 0.898647837057
[[1330  309]
 [ 300 1361]]


Compute correlation of each variable with outcome variable. Select top 100 highest positively-correlated variables to use as features.

Model performs worse now than when all variables are used as features. This is expected because outcome is a function of all 1000 variables in true relationship, so predictive power of 100 variables is not as high as 1000 variables.

In [14]:
correlations = []
for i in range(1000):
    corr_coeff = df['x'+str(i + 1)].corr(df.outcome)
    correlations.append((i, corr_coeff))
top_100 = sorted(correlations, key=lambda x: x[1])[-100:]

In [15]:
indices = ["x"+str(x[0]) for x in top_100]

In [16]:
df = df[['intercept'] + indices + ['z', 'pr', 'outcome']]

In [17]:
models = logreg_pipeline(df)

Train test split -- test size = 0.33
Baseline accuracy: 0.5
Accuracy: 0.568787878788
AUC Score: 0.601245048603
[[896 760]
 [663 981]]


## Case 3: Regime change - intercept 
### DGP is stable over time, then the intercept suddenly changes; data leakage through non-temporal train-test split

Generate data for 5 periods with intercept change in period 2  
Dataset is in temporal order by default

In [18]:
frames = []
for i in [0,1]:
    period_df = generate_data(period = i, n_rows = 2000, m_columns = 5, threshold = 0.8, betas = [1,2,3,4,5], intercept = 1)
    frames.append(period_df)
for i in [2,3,4]:
    period_df = generate_data(period = i, n_rows = 2000, m_columns = 5, threshold = 0.8, betas = [1,2,3,4,5], intercept = 2) 
    frames.append(period_df)
df = pd.concat(frames)
df.to_csv('fake_data.csv')

Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...
Generating 2000 observations...


Run pipeline using appropriate temporal train-test split. Models does well.

In [19]:
models = logreg_pipeline(df, temporal=True)

Time series split
Train periods:  [0] Test periods:  [1]
Baseline accuracy: 0.54
Accuracy: 0.9555
AUC Score: 0.99991262111
[[1076   89]
 [   0  835]]
Train periods:  [0 1] Test periods:  [2]
Baseline accuracy: 0.56
Accuracy: 0.939
AUC Score: 0.999934664619
[[1256  122]
 [   0  622]]
Train periods:  [0 1 2] Test periods:  [3]
Baseline accuracy: 0.6
Accuracy: 0.9
AUC Score: 0.999912029813
[[1301    0]
 [ 200  499]]
Train periods:  [0 1 2 3] Test periods:  [4]
Baseline accuracy: 0.62
Accuracy: 0.889
AUC Score: 0.999962059249
[[1339  222]
 [   0  439]]


Put input file through randomization process, then run it through the pipeline. Model performs no better than random (0 true positives), as expected.

In [20]:
df_random = randomize('fake_data.csv', 'idx', ['period', 'outcome']) # creates csv named fake_data_randomized.csv
models = logreg_pipeline(df_random, temporal = True)

... Randomizing column intercept
... Randomizing column x1
... Randomizing column x2
... Randomizing column x3
... Randomizing column x4
... Randomizing column x5
... Randomizing column z
... Randomizing column pr
Time series split
Train periods:  [0] Test periods:  [1]
Baseline accuracy: 0.54
Accuracy: 0.5865
AUC Score: 0.523102464599
[[1133   32]
 [ 795   40]]
Train periods:  [0 1] Test periods:  [2]
Baseline accuracy: 0.56
Accuracy: 0.689
AUC Score: 0.504981822764
[[1378    0]
 [ 622    0]]
Train periods:  [0 1 2] Test periods:  [3]
Baseline accuracy: 0.6
Accuracy: 0.6505
AUC Score: 0.49058554056
[[1301    0]
 [ 699    0]]
Train periods:  [0 1 2 3] Test periods:  [4]
Baseline accuracy: 0.62
Accuracy: 0.7805
AUC Score: 0.509337072929
[[1561    0]
 [ 439    0]]


Now, introduce data leakage by using non-temporal train-test split. Run this through the pipeline. Model performs well, as expected.

In [21]:
models = logreg_pipeline(df)

Train test split -- test size = 0.33
Baseline accuracy: 0.65
Accuracy: 0.969090909091
AUC Score: 0.996620424671
[[2096   54]
 [  48 1102]]


Observe model performance in each period. No obvious trends...

In [22]:
logreg, X_test, y_test, y_pred = models[0]

periods = np.sort(df.period.unique())

test_df = X_test
test_df['y_test'] = y_test
test_df['y_pred'] = y_pred

for i in periods:
    df_i = test_df[test_df.period == i]
    acc = metrics.accuracy_score(df_i.y_test, df_i.y_pred)
    print("Accuracy when test period is {}: {}".format(i, acc))

Accuracy when test period is 0: 0.9939577039274925
Accuracy when test period is 1: 0.9967159277504105
Accuracy when test period is 2: 0.9667149059334298
Accuracy when test period is 3: 0.9327354260089686
Accuracy when test period is 4: 0.9581464872944694


Put the input file through randomization process, randomizing all columns except period and outcome. Again, introduce data leakage into the pipeline by using non-temporal train-test split. 

In [23]:
df_random = randomize('fake_data.csv', 'idx', ['period', 'outcome']) # generates csv named "fake_data_randomized.csv"
models = logreg_pipeline(df_random)

... Randomizing column intercept
... Randomizing column x1
... Randomizing column x2
... Randomizing column x3
... Randomizing column x4
... Randomizing column x5
... Randomizing column z
... Randomizing column pr
Train test split -- test size = 0.33
Baseline accuracy: 0.65
Accuracy: 0.648787878788
AUC Score: 0.576077918704
[[2134   17]
 [1142    7]]


Observe model performance in each period. We can see that this data leakage causes the higher accuracy in later test periods after the data generating process changes, because information about the future has been leaked.

In [24]:
logreg, X_test, y_test, y_pred = models[0]

periods = np.sort(df.period.unique())

test_df = X_test
test_df['y_test'] = y_test
test_df['y_pred'] = y_pred

for i in periods:
    df_i = test_df[test_df.period == i]
    acc = metrics.accuracy_score(df_i.y_test, df_i.y_pred)
    print("Accuracy when test period is {}: {}".format(i, acc))

Accuracy when test period is 0: 0.5446153846153846
Accuracy when test period is 1: 0.593841642228739
Accuracy when test period is 2: 0.7027027027027027
Accuracy when test period is 3: 0.6384615384615384
Accuracy when test period is 4: 0.7619738751814223
